## Estimate how many copies of a game has sold on steam on reviews
Steam sale data is not public.  Historically, game marketing companies have used a metric called the Boxleiter number which was most recently updated in 2018 [3].  In October 2019, Steam introduced a 'would you like to review this?' button directly to a games launch page.  This button's existence would imply that games with higher reviews would also mean that there are more sales.

The sample dataset for the number of people who own a game come from SteamSpy, a third party source.  So any analysis made should be taken with a grain of salt.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/clean_steam_general.csv')

In [ ]:
# correlate release data with number of users

Sources:

[1] https://newsletter.gamediscover.co/p/how-that-game-sold-on-steam-using

[2] http://greyaliengames.com/blog/how-to-estimate-how-many-sales-a-steam-game-has-made/

[3] https://www.gamasutra.com/blogs/JakeBirkett/20180504/317366/Using_Steam_reviews_to_estimate_sales.php